### Build Architecture2
- 레이어가 많아질 경우 어떻게 할 것인가?
- 모든 레이어를 다 작성할 것인지?
- 디버그를 위한 shape 출력문은 어떻게 만들 것인지?
- 여러개의 레이어를 통과시킨 후 merge할 경우

In [9]:
import torch
import torch.nn as nn
import numpy as np

In [105]:
# 임의의 데이터 생성

input_data = torch.tensor(np.random.randint(0,255, (1,3,244,244)))
input_data = input_data.type(torch.float32)

input_data

tensor([[[[ 52.,  21., 197.,  ..., 216.,  90., 182.],
          [ 42.,  82., 237.,  ..., 180.,   2., 185.],
          [ 10.,  79.,  25.,  ...,   1., 241., 142.],
          ...,
          [179.,  54., 221.,  ...,  28., 128.,  19.],
          [125., 213.,  88.,  ..., 197.,   7., 204.],
          [117.,  49., 117.,  ..., 143.,  92., 244.]],

         [[ 63.,  23., 195.,  ..., 127.,  33., 237.],
          [ 17.,  91.,  60.,  ..., 110.,  92.,  20.],
          [152., 250., 198.,  ..., 185., 207., 124.],
          ...,
          [162., 172.,  13.,  ...,  68.,  49.,  26.],
          [198., 133., 113.,  ...,   6., 191., 243.],
          [202., 176.,  38.,  ...,  60., 226., 113.]],

         [[  1., 162.,  83.,  ...,   8., 187., 141.],
          [101.,   9.,   0.,  ..., 245., 185., 202.],
          [ 91.,  24., 217.,  ..., 146., 191.,  20.],
          ...,
          [125., 149., 155.,  ...,  23., 243., 139.],
          [135., 254., 213.,  ..., 175., 202.,  27.],
          [167., 160.,  21.,  ...

In [50]:
class ConvLayer(nn.Module):
    def __init__(self, input_c, n_neuron, kernel_size=3, stride=1, padding=0, activation='ReLU'):
        super(ConvLayer, self).__init__()
        self.conv = nn.Conv2d(input_c, n_neuron, kernel_size, stride, padding)
        # if activation == 'ReLU':
        self.ReLU = nn.ReLU()
        
    def forward(self, x):
        x = self.conv(x)
        # if activation == 'ReLU':
        x = self.ReLU(x)
        print(x.shape)
        return x
        

        

In [51]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = ConvLayer(3,5,3)
        # self.conv2 = ConvLayer(3,7,3)
        # self.conv_1x1 = ConvLayer(7,5,1)
    
    def forward(self, x):
        output1 = self.conv1(x)
        return output1
        # output2 = self.conv2(x)
        # output2 = self.conv_1x1(output2)
        # output3 = output1 + output2
        # print(output3.shape)

In [52]:
model = CNN()
model

CNN(
  (conv1): ConvLayer(
    (conv): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1))
    (ReLU): ReLU()
  )
)

In [53]:
model(input)

TypeError: ignored

### ResNet 구현해보기
- 참조  
    https://towardsdev.com/implement-resnet-with-pytorch-a9fb40a77448

In [93]:
class PrintShape(nn.Module):
    def __init__(self):
        super(PrintShape, self).__init__()

    def forward(self, x):
        print(x.shape)
        return x

In [121]:
from torch.nn.modules.batchnorm import BatchNorm2d
import torch.nn.functional as F
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super(ResBlock, self).__init__()
        if downsample:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.shortcut = nn.Sequential(
                # nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1),
                # nn.BatchNorm2d(out_channels)
            )

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        shortcut = self.shortcut(x)
        # print('shortcut :', shortcut.size())
        x = self.bn1(self.conv1(x))
        x = F.ReLU(x)
        # print('conv1 :', x.size())
        x = self.bn2(self.conv2(x))
        x = F.ReLU(x)
        # print('conv2 :', x.size())
        output = shortcut + x
        # print('output :', x.size())
        return output


In [122]:
class ResNet18(nn.Module):
    def __init__(self, in_channels, resblock, outputs=1000):
        super(ResNet18, self).__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3), # 7x7 kernel을 사용하고 output shape을 1/2로 줄이기 위해서는 padding이 필요
            nn.MaxPool2d(3, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.layer1 = nn.Sequential(
            resblock(64, 64, downsample=False),
            resblock(64, 64, downsample=False)
        )

        self.layer2 = nn.Sequential(
            resblock(64, 128, downsample=True),
            resblock(128, 128, downsample=False)
        )

        self.layer3 = nn.Sequential(
            resblock(128, 256, downsample=True),
            resblock(256, 256, downsample=False)
        )
        
        self.layer4 = nn.Sequential(
            resblock(256, 512, downsample=True),
            resblock(512, 512, downsample=False)
        )

        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(512, 10)
        
    def forward(self, x):
        x = self.layer0(x),
        # print(x.size())
        x = self.layer1(x),
        x = self.layer2(x),
        x = self.layer3(x),
        x = self.layer4(x),
        x = self.gap(x),
        output = self.fc(x)
        # print('output :', output.size())

        return x

In [123]:
from torchsummary import summary

resnet18 = ResNet18(3, ResBlock, outputs=1000)
summary(resnet18, input_data)

TypeError: ignored

### 후기

TypeError: conv2d() received an invalid combination of arguments - got (tuple, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:  
이거 왜 자꾸 나오는거냐..